<a href="https://colab.research.google.com/github/zuber129/aimlops-colab-worksheets/blob/main/M2_AST_01_Convolutional__Neural_Networks_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Programme in AI and MLOps
## A programme by IISc and TalentSprint
### Assignment: CNN - MNIST, Cats & Dogs

## Learning Objectives:

At the end of the experiment, you will be able to:

1. Understand Conv2D and MaxPooling layers used in ConVNet
2. Build a simple ConvNet for image classification using the digit MNIST dataset
3. Build another  ConvNet for  image Classification using the cats-and-dogs dataset
4. Apply Data Augmentation





## 1. Building a simple CNN


Let's start with a simple example. We will
1. Build a CNN with convolution and pooling layers
2. Train it on the MNIST dataset

The figure below is a typical ConvNet (LeNet) architecture that we are going to build but with different numbers and sizes of filters.

![picture](https://drive.google.com/uc?export=view&id=1vT8e59AYTFRlrrI3C-iUHTctxyhfBiJJ)

### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2237305" #@param {type:"string"}

In [ ]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "M2_AST_01_Convolutional_Neural_Networks_C" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")

    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aimlops-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



In [ ]:
#@title Download Data
!wget -qq https://cdn.iisc.talentsprint.com/AIandMLOps/Datasets/housing_dataset.csv
print("Data downloaded successfully!")

## Import libraries

In [ ]:
# import libraries
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

## Load data

In [ ]:
# Load data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(f"train_images.shape = {train_images.shape}")

## Reshape and convert to float

In [ ]:
# Reshape and convert to float
train_images = train_images.reshape((60000, 28, 28, 1)) # Q: How many samples do we have? A: 60000
train_images = train_images.astype("float32") / 255 # Q: Why are we dividing by 255? A: To rescale data to lie in [0,1]
print(f"train_images.shape = {train_images.shape}")

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

### Building the architecture
LeNet-5: Example of an early ConvNet

In [ ]:
# Define convnet
# Q: Which API are we using? A: Functional API
inputs = keras.Input(shape=(28, 28, 1))     # Q: How many channels does the input image have? A: 1
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)  # Q: Meaning of each argument?
x = layers.MaxPooling2D(pool_size=2)(x)             # Q: What is the height and width of feature maps after this layer? A: 13x13
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)  # Need Dense layer at the end for classification
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()
# Q: Verify no. of params in 1st conv layer # A: (3X3X1 + 1) X 32 ; 32 filters, 9 weights per kernel, 1 bias ber kernel

In [ ]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",     # Q: Why sparse_cat_crossent?  A: labels are not one-hot-encoded
              metrics=["accuracy"])

#### Call Back Function

In [ ]:
# Define a function to return a commmonly used callback_list
def def_callbacks(filepath, mod_chk_mon = "val_loss", tensorboard = True, earlystop = 0 ):
    callback_list = []

    # Defualt callback
    callback_list.append(keras.callbacks.ModelCheckpoint(filepath,
                                         save_best_only = True,
                                         monitor=mod_chk_mon))
    if tensorboard:
      log_dir = "tensorLog_" + filepath
      callback_list.append(keras.callbacks.TensorBoard(log_dir=log_dir))

    if earlystop>0:
       callback_list.append(keras.callbacks.EarlyStopping(patience=earlystop))

    return callback_list

### Training & Evaluation

In [ ]:
PARTIAL_RUN = False
epochs = 10
if PARTIAL_RUN:
  epochs = 2
model.fit(train_images, train_labels, epochs=epochs, validation_split=0.2, batch_size=64, callbacks=def_callbacks("prob1"))

### Prediction on test data

In [ ]:
# Evaluate test accuracy
test_loss, test_acc = model.evaluate(test_images, test_labels)  # Q: Which state is this model at? A: Slightly overfit, trained till 10th epoch
print(f"Test accuracy: {test_acc:.3f}")

Nearly **99%** accurate! This is much better than what we achieved with our feedforward network with only dense layers!

### Tensorboard results

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/tensorLog_prob1

### Now, let's see the importance of pooling layers.

We will make a new model called 'model_no_max_pool' without any pooling layers and compare it with the previous model.

In [ ]:
# Define a new convnet without any pooling layers
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)
# Q: Do you expect more/less no. of trainable params? A: More number of params because of lager no. of neurons before the dense layer

In [ ]:
model_no_max_pool.summary()

#### Observations from models with and without pooling layers
1. The initial model with pooling layers had just 104,202 parameters but the model without pooling layers (model_no_max_pool) has 712,202 parameters i.e. increase in the number of trainable parameters. **Model with pooling layer is less prone to overfitting** due to a smaller number of parameters/weights.

2. What other advantage does the pooing layer provide? **They facilitate learning a spatial hierarchy of features**.

  In the CNN given below. Imagine a 1x1 patch on a C3 feature map. It contains information from 6x6 window of the input layer. On the other hand, if no pooling layers are present, then it would contain information from a 3x3 window in the input layer.


![picture](https://drive.google.com/uc?export=view&id=1vT8e59AYTFRlrrI3C-iUHTctxyhfBiJJ)

**Optional Exercise:** Train the model_no_max_pool
 with the MNIST data set and compare its accuracy with the first model.

**In class exercise:** Let's try to build a LeNet-5 architecure as given in above diagram right now:


* 1st Conv and 2nd Conv layers have a 3x3 & 5x5 kernel respectively
* Pooling layers have 2x2 kernel
* All activations as 'relu' except for last



In [ ]:
inputs = keras.Input(shape=(28,28 , 1))
x = layers.Conv2D(filters= 6, kernel_size= 3, padding= 'same',  activation='relu')(inputs)  # xxx is an argument
x = layers.MaxPooling2D(pool_size= 2)(x)             # infer from diagram
x = layers.Conv2D(filters= 16, kernel_size= 5, padding='valid' , activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(120, activation='relu')(x)
x = layers.Dense(84, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()


## 2. Image Classification


Now, we know how to build a simple CNN, let's build and train one to solve an image classification problem.

We will work with the cats-vs-dogs dataset to classify whether a given image is that of a cat or a dog .i.e a  binary classification problem.

### Import libraries

In [ ]:
import os
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import numpy as np
from tensorflow.keras.utils import image_dataset_from_directory

We have already uploaded the dataset into structured folders. You simply need to download it from our repository.

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/AIandMLOps/Datasets/cats_vs_dogs_small.zip
!unzip -qq '/content/cats_vs_dogs_small.zip'

In [ ]:
# defining path names for futur use
data_dir = '/content/cats_vs_dogs_small'

train_path = data_dir + '/train'
validation_path = data_dir + '/validation'
test_path = data_dir + '/test'

In [ ]:
# Visualise the original data
dog_image = train_path + '/dog/' +  'dog.443.jpg'
print("shape of the dog image is:",imread(dog_image).shape)
plt.imshow(imread(dog_image))

dim1 = []
dim2 = []

for image_file in os.listdir(train_path+'/dog'):
    img = imread(train_path +'/dog/'+image_file)
    d1,d2,colour_channels = img.shape
    dim1.append(d1)
    dim2.append(d2)

sns.jointplot((dim1,dim2))
print("Mean across height of all dog images in train set is:",np.mean(dim1))
print("Mean across width of all dog images in train set is:",np.mean(dim2))
# Q: Do all the images in the dataset have the same sizes? A: No

### Converting the image dataset into a workable format

We have the images in folders. We need to make it into a workable dataset:
  * Which has labels
  * All the images have the same size

For this, we will use the utility [**image_dataset_from_directory**](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory).

Calling image_dataset_from_directory(main_directory, labels='inferred') will return a tf.data.Dataset that yields batches of images from the subdirectories class_a and class_b, together with labels 0 and 1 (0 corresponding to class_a and 1 corresponding to class_b).

In [ ]:
train_dataset = image_dataset_from_directory(
               train_path,
                image_size=(180, 180), # Resize the images to (180,180)
                batch_size=32)
validation_dataset = image_dataset_from_directory(
                      validation_path,
                      image_size=(180, 180),
                      batch_size=32)
test_dataset = image_dataset_from_directory(
                test_path,
                image_size=(180, 180),
                batch_size=32)


In [ ]:
print(f"train_dataset = {train_dataset}")

In [ ]:
# Verify batch size
for data_batch, labels_batch in train_dataset:
  print("data batch shape:", data_batch.shape)
  print("labels batch shape:", labels_batch.shape)
  break
# Q: What is the batch size of each mini-batch? A: 32

In [ ]:
# Define covnet model
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)  # Rescale input to lie between 0 and 1
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # Q: Why sigmoid? A: Binary classification
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# compile the model
model.compile(loss="binary_crossentropy",   # Q: Why binary_crossentropy? A: Binary classification
                      optimizer="rmsprop",
                      metrics=["accuracy"])

In [ ]:
# fit the model
PARTIAL_RUN = False
epochs = 10
if PARTIAL_RUN:
  epochs = 2
history = model.fit(train_dataset,epochs=epochs,validation_data=validation_dataset,callbacks=def_callbacks("convnet_from_scratch.keras"))
## Using a previously defined callback function

In [ ]:
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/tensorLog_convnet_from_scratch.keras

Great! From just **2000** images, our network has learnt to classify images of cats and dogs with an accuracy of apx **70%**

## 3. Data Augmentation

The small dataset can cause a high variance estimation of model performance

Q: How to overcome this and get a more robust model?

Now, we want to avoid this problem altogether by artificially (and cleverly) producing new data from the already available data.

For this, we perform **data augmentation**.

Data augmentation is another regularization method. What other methods did we see in the last tutorial?

Data augmentation takes the approach of generating more training data from existing training samples by augmenting the samples via a number of random transformations that yield a believable-looking image. Common transformations include:
  * Flipping the image
  * Rotating the image
  * Zooming in/out of the image

See some sample images below after augmentation:

![picture](https://drive.google.com/uc?export=view&id=1HRhsHEHtcVptNVMF1EbCGiZX5XuTdrs5)

In [ ]:
# Performing the data augmentation as series of transformations
def get_data_augmented(flip="horizontal",rotation=0.1,zoom=0.2):
    data_augmentation = keras.Sequential([
      keras.layers.RandomFlip(flip),
      keras.layers.RandomRotation(rotation),
      keras.layers.RandomZoom(zoom)])
    return data_augmentation
# Q: what does the above function return? A: A sequence of layers

data_augmentation = get_data_augmented()


In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
# Augmenting data - Transformations of images by random factors
# so the the network never sees the same data twice
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)     # Q: Dropout is a _______ method
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
PARTIAL_RUN = False
epochs = 80
if PARTIAL_RUN:
  epochs = 2
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=validation_dataset,
    callbacks=def_callbacks("convnet_from_scratch_with_augmentation_keras"))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/tensorLog_convnet_from_scratch_with_augmentation_keras

In [ ]:
test_model = keras.models.load_model(
            "convnet_from_scratch_with_augmentation_keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

With data augmentation, we roughly get **82-85%** accuracy. This is a big improvement over the previous approach, where we got roughly 70% accuracy.

### Please answer the questions below to complete the experiment:




In [ ]:
#@title  We are applying 2 convolution filters of size 3X3 on an image of a size 6X6 Pixel having 3 channels. What is the shape of the output after the convolution operation and what is the number of parameters including bias? Given, no padding and stride is one. {run: "auto", form-width: "500px", display-mode: "form" }
Answer = "" #@param ["", "(4X4X2); 56", "(3X3X2); 20", "(3X3X2); 56", "(4X4X2); 20"]

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")